In [ ]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,082 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,373 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jammy-security/m

In [14]:
from pyspark.sql import SparkSession
import cv2
import os

# Initialize Spark Session
spark = SparkSession.builder.appName("VideoFrameExtractor").getOrCreate()

# Function to count every 20th frame in a video
def count_every_20th_frame(video_path):
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    selected_frame_count = 0  # Counter for every 20th frame
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if frame_count % 20 == 0:  # Check if it's the 20th frame
            selected_frame_count += 1
        frame_count += 1
    cap.release()
    return selected_frame_count

# Function to process each folder of videos
def process_folder(folder_path):
    folder_name = os.path.basename(folder_path)
    video_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith(('.mp4', '.avi'))]
    # Sum the counts of every 20th frame from each video in the folder
    total_selected_frames = sum(count_every_20th_frame(video) for video in video_files)
    return (folder_name, total_selected_frames)

# List of folders to process
folders = ["Abuse", "Arson", "Arrest","Burglary","Assault","Explosion","Fighting","Shoplifting","Stealing","Vandalism","Robbery","RoadAccident","Shooting","Banner"]
folder_paths = [os.path.join("/content", folder) for folder in folders]  # Change "/content" to your dataset path

# Create an RDD for folders
folders_rdd = spark.sparkContext.parallelize(folder_paths, numSlices=len(folder_paths))

# Map the folder processing function to the RDD
folder_frame_counts = folders_rdd.map(process_folder)

# Collecting results
results = folder_frame_counts.collect()

# Print results
for result in results:
    print(f"Folder: {result[0]}, Total Selected Frame Count: {result[1]}")

# Stop Spark Session
spark.stop()



Folder: Abuse, Total Selected Frame Count: 8347
Folder: Arson, Total Selected Frame Count: 5545
Folder: Arrest, Total Selected Frame Count: 4837
Folder: Burglary, Total Selected Frame Count: 10890
Folder: Assault, Total Selected Frame Count: 4632
Folder: Explosion, Total Selected Frame Count: 10460
Folder: Fighting, Total Selected Frame Count: 16742
Folder: Shoplifting, Total Selected Frame Count: 8664
Folder: Stealing, Total Selected Frame Count: 6688
Folder: Vandalism, Total Selected Frame Count: 7261
Folder: Robbery, Total Selected Frame Count: 6356
Folder: RoadAccident, Total Selected Frame Count: 3481
Folder: Shooting, Total Selected Frame Count: 13565
Folder: Banner, Total Selected Frame Count: 1900
